In [51]:
import numpy as np
import pandas as pd

df = pd.read_csv("AI.csv",header=0)

D:\Programs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (21,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
df.head()

,Unnamed: 0,AI_DATE,AC_MODEL,AC_SN,OPERATOR_CODE,EVENT_RECORD_IDENTIFIER,DIVERTED_LANDING,AIR_TURN_BACK,REJECTED_TAKE_OFF,GROUND_TURN_BACK,...,ATA_CAUSE,INTERRUPTION_REASON,CORRECTIVE_ACTION,AI_ID,EDIT_DATE,EDITED_BY,Unnamed: 23,ATA_CHAPTER,ATA_CHAPTER_CAUSE,AI_DATE_1
0,0,29-Mar-20,CRJ700,15299,EDV,NaN,0,N,N,Y,...,308102,ICE DET (X) FAIL - STATUS,DEFERRED - REPLACED ICE DETECTOR,590931,13-May-20,T12540,NaN,30,30,20
1,1,30-Mar-20,CRJ700,15125,EDV,NaN,0,N,N,Y,...,243101,TRU FAN FAIL - STATUS,DEFERRED - REPLACED TRU,590932,13-May-20,T12540,NaN,24,24,20
2,2,30-Mar-20,CRJ700,10176,EDV,NaN,0,N,N,Y,...,361315,ISOL FAIL - CAUTION,REPLACED ISOLATION VALVE,590933,13-May-20,T12540,NaN,36,36,20
3,3,3-Apr-20,CRJ700,15046,JZA,NaN,0,N,N,Y,...,302201,Incident: RTRN L COWL ANTI ICE CAUTION MSG. L ...,(Resolved On W/O: 684248 Taskcard: NR-00001 ) ...,591076,27-May-20,T12540,NaN,30,30,20
4,4,8-Apr-20,CRJ700,15044,JZA,NaN,0,N,N,Y,...,213108,Incident: RTRN OVBD COOL CAUTION MESSAGE DISPL...,(Resolved On W/O: 684350 Taskcard: NR-00001 ) ...,591077,27-May-20,T12540,NaN,21,21,20


In [52]:
len(df)

51781

In [80]:
df = df.dropna(axis=0, subset=['ATA_SYMPTOM'])
len(df)

22962

In [54]:
df['ATA_SYMPTOM'].nunique()
df["ATA_SYMPTOM"] = df["ATA_SYMPTOM"].astype(str)

#filtered_df = df[df['ATA_SYMPTOM'].isnull()]

In [42]:
#df = df.astype({"ATA_SYMPTOM": 'str', "ATA_CAUSE": 'str'}) 

#df.filter(items=['ATA_SYMPTOM','ATA_CAUSE',"INTERRUPTION_REASON",
#"CORRECTIVE_ACTION",'ATA_CHAPTER',"ATA_CHAPTER_CAUSE"])


In [91]:
df['features']

0        ICE DET (X) FAIL - STATUS - DEFERRED - REPLACE...
1          TRU FAN FAIL - STATUS - DEFERRED - REPLACED TRU
2           ISOL FAIL - CAUTION - REPLACED ISOLATION VALVE
3        Incident: RTRN L COWL ANTI ICE CAUTION MSG. L ...
4        Incident: RTRN OVBD COOL CAUTION MESSAGE DISPL...
                               ...                        
51776    Pilot Base Heat caution message just after tak...
51777    Attempted to retract gear on departure, gear d...
51778    Tat probe heat, R Aoa heat, r static heat, R p...
51779    After departure, NOSE DOOR OPEN and GEAR DISAG...
51780    APU OIL TEMP, APU OIL PRESS CAUTION MESSAGE IN...
Name: features, Length: 22962, dtype: object

In [92]:
from sklearn.model_selection import train_test_split
#only use feature length and punct
#X = df[['INTERRUPTION_REASON','CORRECTIVE_ACTION']]
#X = df['INTERRUPTION_REASON']+df['CORRECTIVE_ACTION']
df['features'] = df['INTERRUPTION_REASON'].str.cat(df['CORRECTIVE_ACTION'], sep='  ',na_rep=' ')
X = df['features']
y = df['ATA_SYMPTOM']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#X_train

In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',CountVectorizer()),('clf',LinearSVC())])
text_clf.fit(X_train,y_train)

D:\Programs\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('tfidf', CountVectorizer()), ('clf', LinearSVC())])

In [133]:
predictions = text_clf.predict(X_test)

In [135]:
from sklearn import metrics
metrics.confusion_matrix(y_test,predictions)

array([[205,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0, 213,   0],
       [  0,   0,   0, ...,   0,   2,   0]], dtype=int64)

In [134]:
metrics.accuracy_score(y_test,predictions)

0.7147004486671945

In [129]:
from sklearn.naive_bayes import MultinomialNB
#nb_model = MultinomialNB()
#nb_model = Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())])
#nb_model.fit(X_train,y_train)


#vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
counts = vectorizer.fit_transform(X_train.values)
classifier = MultinomialNB()
targets = y_train.values
classifier.fit(counts, targets)

MultinomialNB()

In [130]:
example_counts = vectorizer.transform(X_test)
predictions = classifier.predict(example_counts)
metrics.confusion_matrix(y_test,predictions)

array([[205,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   1,   0],
       [  0,   0,   0, ...,   0, 242,   0],
       [  0,   0,   0, ...,   0,   2,   0]], dtype=int64)

In [131]:
metrics.accuracy_score(y_test,predictions)

0.35009237265769333

In [101]:
from sklearn.linear_model import LogisticRegression
lr_model = Pipeline([('tfidf',TfidfVectorizer()),('lr',LogisticRegression())])
lr_model.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('lr', LogisticRegression())])

In [104]:
predictions = lr_model.predict(X_test)
metrics.confusion_matrix(y_test,predictions)

array([[205,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   1,   0],
       [  0,   0,   0, ...,   0, 241,   0],
       [  0,   0,   0, ...,   0,   2,   0]], dtype=int64)

In [105]:
metrics.accuracy_score(y_test,predictions)

0.7158880971232515

In [122]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='poly', degree=3)
sv_model = Pipeline([('tfidf',TfidfVectorizer()),('lr',svclassifier)])
sv_model.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('lr', SVC(kernel='poly'))])

In [123]:
predictions = sv_model.predict(X_test)
#metrics.confusion_matrix(y_test,predictions)

In [124]:
metrics.accuracy_score(y_test,predictions)

0.5050145157033518

In [114]:
svclassifier = SVC(kernel='rbf')
sv_model = Pipeline([('tfidf',TfidfVectorizer()),('lr',svclassifier)])
sv_model.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('lr', SVC())])

In [115]:
predictions = sv_model.predict(X_test)
metrics.confusion_matrix(y_test,predictions)

array([[205,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0, 244,   0],
       [  0,   0,   0, ...,   0,   2,   0]], dtype=int64)

In [116]:
metrics.accuracy_score(y_test,predictions)

0.7082343626286619

In [117]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=141)
sv_model = Pipeline([('tfidf',TfidfVectorizer()),('lr',knn)])
sv_model.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('lr', KNeighborsClassifier(n_neighbors=141))])

In [118]:
predictions = sv_model.predict(X_test)
metrics.confusion_matrix(y_test,predictions)

array([[205,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   1,   0],
       [  0,   0,   0, ...,   0, 256,   0],
       [  0,   0,   0, ...,   0,   2,   0]], dtype=int64)

In [119]:
metrics.accuracy_score(y_test,predictions)

0.6508313539192399